# Download and store STOOQ data

This notebook contains information on downloading the STOOQ stock and ETF price data that we use in [Chapter 09](../09_time_series_models) for a pairs trading strategy based on cointegration and [Chapter 11](../11_decision_trees_random_forests) for a long-short strategy using Random Forest return predictions.

## Imports & Settings

In [1]:
!pip install ipython-autotime
!pip install pandas_datareader

     |████████████████████████████████| 107 kB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 5.5 MB 9.7 MB/s eta 0:00:01


In [2]:
# hdf failed
#!conda install -y -c anaconda pytables
#!conda install -c ska tables

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from pathlib import Path
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from sklearn.datasets import fetch_openml

pd.set_option('display.expand_frame_repr', False)

## Set Data Store path

Modify path if you would like to store the data elsewhere and change the notebooks accordingly

In [5]:
%load_ext autotime
### set path
import os
from pathlib import Path
path_home = os.getcwd()
path_home = Path(path_home)
path_data = path_home / 'data'
if not path_data.exists():
    path_data.mkdir()
#path_result = path_home / 'result'

time: 1.69 ms (started: 2021-04-23 14:05:31 +00:00)


## Stooq Historical Market Data
##### several cells made to one as stooq folder makes notebook freezing
##### seems memory intensive

In [6]:
### Download price data
# 1. Download **price data** for the selected combination of asset class, market and frequency from [the Stooq website](https://stooq.com/db/h/)
# 2. Store the result under `stooq` using the preferred folder structure outlined on the website. It has the structure: `/data/freq/market/asset_class`, such as `/data/daily/us/nasdaq etfs`.

stooq_path = path_home / 'stooq'
if not stooq_path.exists():
    stooq_path.mkdir()

STOOQ_URL = 'https://static.stooq.com/db/h/'

for market in ['jp', 'us']:
    data_url = f'd_{market}_txt.zip'
    response = requests.get(STOOQ_URL + data_url).content
    with ZipFile(BytesIO(response)) as zip_file:
        for i, file in enumerate(zip_file.namelist()):
            if not file.endswith('.txt'):
                continue
            local_file = stooq_path / file
            local_file.parent.mkdir(parents=True, exist_ok=True)
            with local_file.open('wb') as output:
                for line in zip_file.open(file).readlines():
                    output.write(line)

### Add symbols
# Add the corresponding **symbols**, i.e., tickers and names by following the directory tree on the same site. You can also adapt the following code snippet using the appropriate asset code that you find by inspecting the url; this example works for NASDAQ ETFs that have code `g=69`:
# ```python
# df = pd.read_csv('https://stooq.com/db/l/?g=69', sep='        ').apply(lambda x: x.str.strip())
# df.columns = ['ticker', 'name']
# df.drop_duplicates('ticker').to_csv('stooq/data/tickers/us/nasdaq etfs.csv', index=False)
# ```

metadata_dict = {
    ('jp', 'tse etfs'): 34,
    ('jp', 'tse stocks'): 32,
    ('us', 'nasdaq etfs'): 69,
    ('us', 'nasdaq stocks'): 27,
    ('us', 'nyse etfs'): 70,
    ('us', 'nyse stocks'): 28,
    ('us', 'nysemkt stocks'): 26
}

for (market, asset_class), code in metadata_dict.items():
    df = pd.read_csv(f'https://stooq.com/db/l/?g={code}', sep='        ').apply(lambda x: x.str.strip())
    df.columns = ['ticker', 'name']
    df = df.drop_duplicates('ticker').dropna()
    print(market, asset_class, f'# tickers: {df.shape[0]:,.0f}')
    path = stooq_path / 'tickers' / market
    if not path.exists():
        path.mkdir(parents=True)
    df.to_csv(path / f'{asset_class}.csv', index=False)    
    
    
### Store price data
# It removes files that do not have data or do not appear in the corresponding list of symbols.
# The function `get_stooq_prices_and_symbols` loads data assuming the directory structure described above and takes the following arguments:
# - frequency (see Stooq website for options as these may change; default is `daily`
# - market (default: `us`), and 
#- asset class (default: `nasdaq etfs`.

def get_stooq_prices_and_tickers(frequency='daily',
                                 market='us',
                                 asset_class='nasdaq etfs'):
    prices = []
    
    tickers = (pd.read_csv(stooq_path / 'tickers' / market / f'{asset_class}.csv'))

    if frequency in ['5 min', 'hourly']:
        parse_dates = [['date', 'time']]
        date_label = 'date_time'
    else:
        parse_dates = ['date']
        date_label = 'date'
    names = ['ticker', 'freq', 'date', 'time', 
             'open', 'high', 'low', 'close','volume', 'openint']
    
    usecols = ['ticker', 'open', 'high', 'low', 'close', 'volume'] + parse_dates
    path = stooq_path / 'data' / frequency / market / asset_class
    print(path.as_posix())
    files = path.glob('**/*.txt')
    for i, file in enumerate(files, 1):
        if i % 500 == 0:
            print(i)
        if file.stem not in set(tickers.ticker.str.lower()):
            print(file.stem, 'not available')
            file.unlink()
        else:
            try:
                df = (pd.read_csv(
                    file,
                    names=names,
                    usecols=usecols,
                    header=0,
                    parse_dates=parse_dates))
                prices.append(df)
            except pd.errors.EmptyDataError:
                print('\tdata missing', file.stem)
                file.unlink()

    prices = (pd.concat(prices, ignore_index=True)
              .rename(columns=str.lower)
              .set_index(['ticker', date_label])
              .apply(lambda x: pd.to_numeric(x, errors='coerce')))
    return prices, tickers


### We'll be using US equities and ETFs in [Chapter 9](../09_time_series_models) and and Japanese equities in [Chapter 11](../11_decision_trees_random_forests). The following code collects the price data for the period 2000-2019 and stores it with the corresponding symbols in the global `assets.h5` store:
# load some Japanese and all US assets for 2000-2019
markets = {'jp': ['tse stocks'],
           'us': ['nasdaq etfs', 'nasdaq stocks', 'nyse etfs', 'nyse stocks', 'nysemkt stocks']
          }
frequency = 'daily'

idx = pd.IndexSlice
for market, asset_classes in markets.items():
    for asset_class in asset_classes:
        print(f'\n{asset_class}')
        prices, tickers = get_stooq_prices_and_tickers(frequency=frequency, 
                                                       market=market, 
                                                       asset_class=asset_class)
        
        prices = prices.sort_index().loc[idx[:, '2000': '2019'], :]
        names = prices.index.names
        prices = (prices
                  .reset_index()
                  .drop_duplicates()
                  .set_index(names)
                  .sort_index())
        
        print('\nNo. of observations per asset')
        print(prices.groupby('ticker').size().describe())
        key = f'stooq_{market}_{asset_class.replace(" ", "_")}_'
        
        print(prices.info(null_counts=True))
        fn = key + 'prices.csv'
        prices.to_csv(path_data / fn)
        
        print(tickers.info())
        fn = key + 'tickers.csv'
        tickers.to_csv(path_data / fn)

        
### rm stooq dir which make notebook freezed
!rm -r stooq

jp tse etfs # tickers: 341
jp tse stocks # tickers: 3,773
us nasdaq etfs # tickers: 405
us nasdaq stocks # tickers: 4,187
us nyse etfs # tickers: 1,822
us nyse stocks # tickers: 3,397
us nysemkt stocks # tickers: 292

tse stocks
/home/jupyter/random_forests/stooq/data/daily/jp/tse stocks
500
1000
1500
2000
2500
3000
3500

No. of observations per asset
count    3623.000000
mean     2805.231852
std      1177.353004
min         1.000000
25%      2147.500000
50%      3041.000000
75%      3621.000000
max      4905.000000
dtype: float64
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10163355 entries, ('1301.JP', Timestamp('2005-03-22 00:00:00')) to ('9997.JP', Timestamp('2019-12-30 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count     Dtype  
---  ------  --------------     -----  
 0   open    10163355 non-null  float64
 1   high    10163355 non-null  float64
 2   low     10163355 non-null  float64
 3   close   10163355 non-null  float64
 4   volume  10163355 non-nu

In [7]:
!zip data_stooq.zip data/*

  adding: data/stooq_jp_tse_stocks_prices.csv (deflated 74%)
  adding: data/stooq_jp_tse_stocks_tickers.csv (deflated 58%)
  adding: data/stooq_us_nasdaq_etfs_prices.csv (deflated 71%)
  adding: data/stooq_us_nasdaq_etfs_tickers.csv (deflated 70%)
  adding: data/stooq_us_nasdaq_stocks_prices.csv (deflated 70%)
  adding: data/stooq_us_nasdaq_stocks_tickers.csv (deflated 63%)
  adding: data/stooq_us_nyse_etfs_prices.csv (deflated 70%)
  adding: data/stooq_us_nyse_etfs_tickers.csv (deflated 71%)
  adding: data/stooq_us_nyse_stocks_prices.csv (deflated 69%)
  adding: data/stooq_us_nyse_stocks_tickers.csv (deflated 62%)
  adding: data/stooq_us_nysemkt_stocks_prices.csv (deflated 73%)
  adding: data/stooq_us_nysemkt_stocks_tickers.csv (deflated 55%)
time: 1min 53s (started: 2021-04-23 14:13:33 +00:00)


In [8]:
!gsutil cp data_stooq.zip gs://finance_data_bucket

Copying file://data_stooq.zip [Content-Type=application/zip]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1 files][386.2 MiB/386.2 MiB]                                                
Operation completed over 1 objects/386.2 MiB.                                    
time: 5.95 s (started: 2021-04-23 14:15:27 +00:00)
